# O nelinearnoj regresiji

## Kvadratna regresija

U prethodna dva potpoglavlja smo uvidjeli dvije pogodnosti linearne regresije:
    - Brza i jednostavna implementacija
    - Tentativno prihvatljiva tačnost
Stoga nam je ovaj algoritam koristan kada nam je potrebna neka brza i donekle kvalitetna analiza podataka. No, za neku ozbiljniju analizu podataka ili za neki precizniji model predikcije, linearna regresija je praktički neupotrebljiva. Npr. analizirajmo linearnu regresiju koju smo izračunali u prvom poglavlju.

In [1]:
from functools import partial

import numpy as np


def calculate_line_height(x: float, k: float, n: float) -> float:
    """
    Returns:
        Hight of line y = k*x + n for given k, x and n parameters.
    """
    return k * x + n

def calculate_linear_regression_mse(
    training_set: np.ndarray, m_0: float, m_1: float) -> float:
    """
    Returns:
        Mean squared error produced on top of provided training set,
        using linear regression y = m_1*x + m_0
    """
    inputs: np.ndarray = training_set.T[0]
    outputs: np.ndarray = training_set.T[1]
    linear_regression_predict: Callable[[float], float] = partial(
        calculate_line_height, k=m_1, n=m_0)
    
    predictions: np.ndarray = linear_regression_predict(inputs)
    squared_errors = (predictions - outputs) ** 2
    
    return np.mean(squared_errors)

In [2]:
from typing import List, Tuple

import plotly.offline as ply
import plotly.graph_objs as go


# Instantiate dataset
car_mileage_vs_value_data_set: List[List[int]] = [
    (10000, 31000), (400000, 19000), (5000, 32000), (0, 40000),
    (1000, 33000), (100000, 26000), (50000, 29000), (50, 35000),
    (20000, 30000), (200000, 20000)]
car_mileage_vs_value_data_set: np.ndarray = np.array(car_mileage_vs_value_data_set)
car_mileages: np.ndarray = car_mileage_vs_value_data_set.T[0]
car_values: np.ndarray = car_mileage_vs_value_data_set.T[1]

# Instantiate plot.ly objects
ply.init_notebook_mode(connected=True)

plot_data = go.Scatter(x=car_mileages, 
                       y=car_values, 
                       mode='markers',
                       marker=dict(color='black'),
                       name='Training point')

layout = go.Layout(xaxis=dict(ticks='', showticklabels=True,
                              zeroline=False),
                   yaxis=dict(ticks='', showticklabels=True,
                              zeroline=False),
                   showlegend=False, hovermode='closest')

# Calculate linear regression
assert len(car_mileages) == len(car_values), 'Invalid train/labels size'
training_set_size: int = len(car_values)


m_0_ = np.sum(car_mileages ** 2)
m_1_ = np.dot(car_mileages, car_values)
    
x_ = np.mean(car_mileages)
y_ = np.mean(car_values)
    
x__ = m_0_ / training_set_size / x_
y__ = m_1_ / training_set_size / x_

m_1: float = (y__ - y_) / (x__ - x_)
m_0: float = y_ - m_1 * x_
linear_regression_mse: float = calculate_linear_regression_mse(training_set=car_mileage_vs_value_data_set, m_0=m_0, m_1=m_1)
print(f'MSE: {linear_regression_mse}')

# Construct regression line
calculus_linear_regression_line = go.Scatter(x=car_mileages, 
                                             y=car_mileages * m_1 + m_0,
                                             mode='lines',
                                             line=dict(color='green', width=3),
                                             name='Regression line')

# Plot both data and regression line
figure = go.Figure(data=[plot_data,
                         calculus_linear_regression_line],
                   layout=layout)

ply.iplot(figure)

MSE: 10080774.695512515


Dobiveni MSE je za neke primjene praktički neprihvatljiv. Srednja greška cijene auta od $\sqrt{MSE}=\sqrt{10080774}\$=3175\$$ je većini kupaca prevelika.

Npr. u našem trening skupu stoji da auto koje je prešlo 200.000km košta 20.000\\$, dok naša linearna regresija procjenjuje isto auto na 24.335\\$ (pokrenite kod iznad za iscrtavanje plota i pređite kursorom miša preko tačke koordinata $(200k, 20k)$, a zatim preko prave regresije tačno iznad nje). U nekim primjenama, ovolika razlika između stvarne i procijenjene vrijednosti ili generalnije kazano, ovolika **greška** našeg **modela predikcije** je neprihvatljiva.

Srećom, postoje načini da se greška našeg modela smanji, ali ih moramo tražiti u promjeni samog modela. Jedna takva ideja leži u odbacivanju linearnosti iz modela regresije. Naime, postavlja se pitanje: Šta će se desiti ukoliko našu pravu linearne regresije, zamijenimo nekom krivom? Da li će ukupna greška modela biti manja? To jeste, u našem slučaju, da li će $MSE$ biti manji? Pogledajmo zajedno!

Prije svega, koju krivu da koristimo? Prijestimo se naše linearne regresije u analitičkom zapisu: $$p: y = m_1\cdot x + m_0$$ Dakle, postavlja se pitanje, kojom krivom da zamijenimo našu pravu $p$. Hajde da probamo sa nekom jednostavnom. Recimo, kvadratnom: $$p_2: y = m_2\cdot x^2 + m_1\cdot x + m_0$$

Tada se naš MSE transformira u $$MSE(m_0, m_1, m_2) = \frac{1}{|\chi|}\cdot\sum_{(x, y) \in \chi}{(m_2\cdot x^2 + m_1\cdot x + m_0 - y)^2}$$ gdje je $\chi$ naš trening skup.

Dakle potrebno je pronaći parametre $m_0$, $m_1$ i $m_2$ takve da je navedeni $MSE$ što manji. Drugim riječima, potrebno je pronaći minimum funkcije više parametara $MSE(m_0, m_1, m_2)$. Srećom po nas, navedena funkcija je diferencijabilna, pa se možemo poslužiti alatima kalkulusa više promjenjivih da nađemo njen minimum.

No, ta sreća ne traje dugo, jer možemo ubrzo uvidjeti da je izračun datog minimuma, posebice rješavanje sistema parcijalnih izvoda, jedna groteska. *Unatoč tome, entuzijastičan student je u mnogome ohrabren da proba sprovesti taj račun, ako ništa, razonode radi.*

Bilo kako bilo, sada se nećemo zamarati samim načinom pronalaska optimalnih parametara $m_0$, $m_1$ i $m_2$, nego ćemo iskoristiti neko postojeće (implementirano) rješenje. Jedno takvo rješenje je metoda `polyfit` biblioteke `numpy`. *Vrijedi pomenuti i to da je nelinearna regresija poznata i po imenu **polinomska regresija**. Otud i naziv metode `polyfit`.*

Metoda `polyfit` prima tri obavezna parametra:
- x: Lista svih x-koordinata trening skupa $\chi$
- y: Lista svih y-koordinata trening skupa $\chi$
- deg: Stepen $n$ polinoma $P[x] = m_n\cdot x^n + m_{n-1}\cdot x^{n-1} + \dots + m_2\cdot x^2 + m_1\cdot x + m_0$ za koji želimo da pronađemo koeficijente $m_i\in \mathbb{R}$, $i=\overline{0,n}$ takve da je $MSE(m_0, m_1, m_2, \dots, m_n) = \frac{1}{|\chi|}\cdot\sum_{(x, y) \in \chi}{(m_n\cdot x^n + m_{n-1}\cdot x^{n-1} + \dots + m_2\cdot x^2 + m_1\cdot x + m_0 - y)^2}$ što je moguće manji.

I vraća upravo niz koeficijenata $m_i\in \mathbb{R}$, $i=\overline{n,0}$ za koje je navedeni $MSE(m_0, m_1, m_2, \dots, m_n)$ najmanji mogući.

Pogledajmo demonstraciju na našem konkretnom trening skupu. Primijetimo da stavljajući da je `deg=2`, metodom polyfit nalazimo upravo koeficijente $m_0$, $m_1$ i $m_2$ naše kvadratne krive $p_2$.

In [3]:
m_2, m_1, m_0 = np.polyfit(
    x=car_mileages,
    y=car_values,
    deg=2)

print(f'm_2={m_2}, m_1={m_1}, m_0={m_0}')

m_2=1.737533245436812e-07, m_1=-0.10732409009949634, m_0=34234.81248740836


Implementirajmo pomoćnu metodu `calculate_polynomial_mse(training_set, *coefficients)` koja za proslijeđen trening skup $\chi=\{(x_i, y_i) | i=\overline{1,k}, k\in\mathbb{N}\}$ i koeficijente $m_i\in \mathbb{R}$, $i=\overline{n,0}$ računa $MSE(m_0, m_1, m_2, \dots, m_n) = \frac{1}{|\chi|}\cdot\sum_{(x, y) \in \chi}{(m_n\cdot x^n + m_{n-1}\cdot x^{n-1} + \dots + m_2\cdot x^2 + m_1\cdot x + m_0 - y)^2}$.
Također, koristimo metodu `poly1d` biblioteke `numpy` kao olakšicu za izračunavanje vrijednosti polinoma $P[x]$. Ona prima jedan obavezan parametar, a to je niz koeficijenata $[m_0, m_1, \dots, m_n]$ i vraća objekat koji je ekvivalent polinomu $P[x] = m_n\cdot x^n + m_{n-1}\cdot x^{n-1} + \dots + m_2\cdot x^2 + m_1\cdot x + m_0$. Taj objekat nam pojednostavljuje izračunavanje datog polinoma $P[x]$ za dato $x$ i koeficijente $m_i\in \mathbb{R}$, $i=\overline{0,n}$. Pogledajmo na koji način to radimo tačno u nastavku.

In [4]:
from typing import List, Tuple


def calculate_polynomial_mse(training_set: List[Tuple[float, float]], *coefficients: float) -> float:
    polynomial = np.poly1d(coefficients)
    
    return sum([(polynomial(x) - y)**2 for x, y in training_set]) / len(training_set)

Izračunajmo $MSE(m_0,m_1,m_2)$ za prethodno pronađene koeficijente naše krive $p_2$ i uporedimo ga sa $MSE$-om dobivenim linearnom regresijom $p$.

In [5]:
quadratic_regression_mse: float = calculate_polynomial_mse(car_mileage_vs_value_data_set, m_2, m_1, m_0)
print(f'Quadratic regression MSE: {quadratic_regression_mse}')
print(f'Linear regression MSE: {linear_regression_mse}')

Quadratic regression MSE: 4816444.122719141
Linear regression MSE: 10080774.695512515


Značajna razlika! Prosječno odstupanje predviđene vrijednosti od stvarne, koristeći kvadratnu krivu, je $\sqrt{MSE}=\sqrt{4816444}\$=2194\$$.

Iscrtajmo pronađenu krivu:

In [7]:
polynomial = np.poly1d([m_2, m_1, m_0])

x_new = np.linspace(min(car_mileages), max(car_mileages), 50)
y_new = polynomial(x_new)

regression_curve = go.Scatter(
    x=x_new,
    y=y_new,
    mode='lines',
    line=dict(color='blue', width=3),
    name='Regression curve')

figure = go.Figure(data=[plot_data,
                         calculus_linear_regression_line,
                         regression_curve],
                   layout=layout)

ply.iplot(figure)

Vidjevši plavu krivu, nam je još jasnije zašto je novodobiveni MSE toliko niže vrijednosti od prethodnog. Naime, plava kriva, kojoj odgovara novi MSE, je uglavnom "bliža" našim trening tačkama od linearne, zelene, prave.

Kriva $p_2$ se još naziva i **regresionom krivom** stepena dva ili **kvadratnom regresionom krivom**, a cijeli koncept kroz koji smo pronašli istu **regresijom drugog stepena** ili **kvadratnom regresijom**.

*Nameće se pitanje, da li možemo dodatno smanjiti MSE ukoliko još dodatno povećamo stepen regresione krive. Šta mislite? Saznati ćemo u narednim vježbama!*